In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
import math

In [ ]:
heads = 8
d = 512 # embedding size
dff = 2049 # expansiondim
N = 6 # layers
p = 0.1 # dropout rate

src = torch.randint(0, 100, (1, 4)) # 100 words in vocab
trg = torch.randint(0, 50, (1, 2)) # 2-word target sencence


In [ ]:
class Embedding(nn.Module):
    # embedding layer with scaling and dropout
    def __init__(self, d : int, vocab_size : int):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d)
            
    def forward(self, x: Tensor) -> Tensor:
        return self.embedding(x)

In [ ]:
e = Embedding(d, 100)
e(src).shape

In [ ]:
class PE(nn.Module):
    # Positional encoding fn with dropout
    # d = embedding size
    # p = prob of dropout
    def __init__(self, d : int, p : int, max_len = 100):
        super().__init__()
        self.pe = torch.zeros(max_len, d)
        pos = torch.arange(0, max_len, 1).unsqueeze(1)
        div = torch.pow(10_000, 2 * torch.arange(0, d, 2) / d)
        
        self.pe[:, 0::2] = torch.sin(pos / div)
        self.pe[:, 1::2] = torch.cos(pos / div)
        
        self.dropout = nn.Dropout(p)
        

In [ ]:
def forward(self, x, Tensor) -> Tensor:
    return self.dropout(x + self.pe[:x.shape[1]])


In [ ]:
class SelfAttention(nn.Module):
    # Multi head self-attention sub-layer
    
    def __init__(self, heads : int, d : int):
        super().__init__()
        
        self.heads = heads
        self.head_dim = d // heads
        self.d = d
        self.Q = nn.Linear(self.head_dim, self.head_dim)
        self.K = nn.Linear(self.head_dim, self.head_dim)
        self.V = nn.Linear(self.head_dim, self.head_dim)

        self.norm = nn.LayerNorm(d)